<a href="https://colab.research.google.com/github/abhaymise/Blood_cells_image_segmentation/blob/master/ai/phased_fine_tuning_imagenet_models_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
import torch.nn as nn
import torch.optim as optim


# Data preprocessing and loading
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Assume you have a dataset (CIFAR-10 in this case)
dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Load pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Freeze the initial layers
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer for the number of classes in your task
num_classes = 10  # CIFAR-10 has 10 classes
model.fc = nn.Linear(model.fc.in_features, num_classes)


# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Function to unfreeze specified layers
def unfreeze_layers(model, start_layer, end_layer):
    for i, param in enumerate(model.parameters()):
        if start_layer <= i <= end_layer:
            param.requires_grad = True

def evaluate_model(model,val_dataloader):
  # Evaluate on the validation set
  model.eval()
  with torch.no_grad():
      correct = 0
      total = 0
      for inputs, labels in val_dataloader:
          outputs = model(inputs)
          _, predicted = torch.max(outputs, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  accuracy = correct / total
  print(f'Accuracy on the validation set: {accuracy * 100:.2f}%')

def train_model(model,train_dataloader,optimizer,criterion,
                num_epochs):
  model.train()
  for epoch in range(num_epochs):
    # model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Training loop
num_epochs = 5

# Phase 1: Train only the last layer
train_model(model,train_dataloader,optimizer,criterion,
                num_epochs)

evaluate_model(model,val_dataloader)
# Phase 2: Unfreeze some more layers and continue training
unfreeze_layers(model, 6, 10)  # Unfreeze layers 6 to 10

train_model(model,train_dataloader,optimizer,criterion,
                num_epochs)
evaluate_model(model,val_dataloader)

# Phase 3: Unfreeze all layers and fine-tune
unfreeze_layers(model, 0, 10)  # Unfreeze all layers

train_model(model,train_dataloader,optimizer,criterion,
                num_epochs)

evaluate_model(model,val_dataloader)

# Fine-tune all layers for a few more epochs
for param in model.parameters():
    param.requires_grad = True

train_model(model,train_dataloader,optimizer,criterion,
                num_epochs)
evaluate_model(model,val_dataloader)

# Freeze all layers except the named onne
for name, param in model.named_parameters():
    if name not in ['classifier.3.weight', 'classifier.3.bias', 'classifier.4.weight', 'classifier.4.bias']:
        param.requires_grad = False

train_model(model,train_dataloader,optimizer,criterion,
                num_epochs)
evaluate_model(model,val_dataloader)